### Running Anomaly Detection with a Trained `ohana` Model

This notebook demonstrates how to load a pre-trained 3D U-Net model and use it to find anomalies in a sample H2RG exposure file. We will follow the core logic found in the `ohana.predict.predictor` module.

We will perform the following steps:
1.  **Set up paths** to the model, configuration, and data.
2.  **Load the `UNet3D` model** and its trained weights.
3.  **Load and preprocess** the exposure data.
4.  **Run patch-based inference** to generate a full prediction mask.
5.  **Extract anomaly locations** from the mask.
6.  **Visualize the results.**

##### Step 1: Imports and Configuration

First, let's import the necessary modules from the `ohana` package and other libraries. We also define the paths to our trained model, the configuration file, and the exposure data we want to analyze.

In [5]:
import torch
import yaml
import numpy as np
import os
from collections import OrderedDict
from tqdm import tqdm
from scipy.ndimage import label, center_of_mass
import matplotlib.pyplot as plt
import sys

In [6]:
sys.path.insert(0, '../')

In [7]:
# Import the necessary classes from your ohana package
# Make sure the 'ohana' directory is in your Python path
from ohana.models.unet_3d import UNet3D
from ohana.preprocessing.data_loader import DataLoader
from ohana.preprocessing.preprocessor import Preprocessor
from ohana.visualization.plotter import ResultVisualizer

In [14]:
# --- Configuration ---
# NOTE: Replace these with the actual paths to your files.
MODEL_PATH = "../trained_models/best_model_unet3d.pth" # Path to your trained model
CONFIG_PATH = "../configs/creator_config.yaml" # Path to the config file used for training
EXPOSURE_PATH = "sample_data/my_exposure.fits" # Path to the exposure file you want to analyze
OUTPUT_DIR = "prediction_outputs"

In [10]:
# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


##### Step 2: Load the Trained Model

Next, we load the `UNet3D` model architecture and the saved weights from your `.pth` file. The code includes a step to handle models that were trained using `nn.DataParallel` on multiple GPUs. This logic comes directly from the `Predictor` class.

In [15]:
# Load the configuration file
with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

# Initialize the model
# The number of classes should match the 'num_classes' in your config
model = UNet3D(n_channels=1, n_classes=config['num_classes'])

# Load the trained weights
print(f"Loading model from: {MODEL_PATH}")
state_dict = torch.load(MODEL_PATH, map_location=device)

# Handle models saved with nn.DataParallel
if next(iter(state_dict)).startswith('module.'):
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:]  # remove `module.`
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
else:
    model.load_state_dict(state_dict)

model.to(device)
model.eval() # Set the model to evaluation mode
print("Model loaded successfully.")

Loading model from: ../trained_models/best_model_unet3d.pth


FileNotFoundError: [Errno 2] No such file or directory: '../trained_models/best_model_unet3d.pth'